# **Fair Housing Project 2022 Draft**

In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-paper')
#import seaborn as sns; sns.set()

from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error
import math

#this will mute errors that make your final pdf less legible
import warnings
warnings.filterwarnings('ignore')

from google.colab import drive
from google.colab import files
drive.mount('/content/drive')
!cp /content/drive/MyDrive/fy2022pa-4.csv .
!cp /content/drive/MyDrive/msamd_14454.csv .

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
prop_type = pd.read_csv('fy2022pa-4.csv')
prop_type.head(20)

,PID,CM_ID,GIS_ID,ST_NUM,ST_NAME,UNIT_NUM,CITY,ZIPCODE,BLDG_SEQ,NUM_BLDGS,...,KITCHEN_STYLE2,KITCHEN_STYLE3,HEAT_TYPE,HEAT_FUEL,AC_TYPE,FIRE_PLACE,ORIENTATION,NUM_PARKING,PROP_VIEW,CORNER_UNIT
0,100001000,,100001000.0,104 A 104,PUTNAM ST,NaN,EAST BOSTON,2128,1.0,1,...,S - Semi-Modern,S - Semi-Modern,W - Ht Water/Steam,NaN,N - None,0,NaN,3,A - Average,NaN
1,100002000,,100002000.0,197,LEXINGTON ST,NaN,EAST BOSTON,2128,1.0,1,...,M - Modern,M - Modern,F - Forced Hot Air,NaN,C - Central AC,0,NaN,0,A - Average,NaN
2,100003000,,100003000.0,199,LEXINGTON ST,NaN,EAST BOSTON,2128,1.0,1,...,S - Semi-Modern,S - Semi-Modern,S - Space Heat,NaN,N - None,0,NaN,0,A - Average,NaN
3,100004000,,100004000.0,201,LEXINGTON ST,NaN,EAST BOSTON,2128,1.0,1,...,S - Semi-Modern,S - Semi-Modern,W - Ht Water/Steam,NaN,N - None,0,NaN,0,A - Average,NaN
4,100005000,,100005000.0,203,LEXINGTON ST,NaN,EAST BOSTON,2128,1.0,1,...,S - Semi-Modern,NaN,W - Ht Water/Steam,NaN,N - None,0,NaN,0,A - Average,NaN
5,100006000,,100006000.0,205 207,LEXINGTON ST,NaN,EAST BOSTON,2128,1.0,1,...,M - Modern,M - Modern,E - Electric,NaN,N - None,0,NaN,0,A - Average,NaN
6,100007000,,100007000.0,209 211,LEXINGTON ST,NaN,EAST BOSTON,2128,1.0,1,...,S - Semi-Modern,S - Semi-Modern,W - Ht Water/Steam,NaN,N - None,0,NaN,0,A - Average,NaN
7,100008000,,100008000.0,213,LEXINGTON ST,NaN,EAST BOSTON,2128,1.0,1,...,S - Semi-Modern,S - Semi-Modern,W - Ht Water/Steam,NaN,N - None,0,NaN,0,A - Average,NaN
8,100009000,,100009000.0,215,LEXINGTON ST,NaN,EAST BOSTON,2128,1.0,1,...,S - Semi-Modern,M - Modern,W - Ht Water/Steam,NaN,N - None,0,NaN,0,A - Average,NaN
9,100010000,,100010000.0,217,LEXINGTON ST,NaN,EAST BOSTON,2128,1.0,1,...,M - Modern,M - Modern,W - Ht Water/Steam,NaN,C - Central AC,0,NaN,0,A - Average,NaN


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
data1 = pd.read_csv('msamd_14454.csv')
data1.head()

,activity_year,lei,derived_msa-md,state_code,county_code,census_tract,conforming_loan_limit,derived_loan_product_type,derived_dwelling_category,derived_ethnicity,...,denial_reason-2,denial_reason-3,denial_reason-4,tract_population,tract_minority_population_percent,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units
0,2021,549300DAUXQ2DCY4H838,14454,MA,25021.0,2.502140e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,4448,31.27,113700,91,1016,1519,72
1,2021,549300DAUXQ2DCY4H838,14454,MA,25021.0,2.502140e+10,NC,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,4230,7.78,113700,179,1228,1394,60
2,2021,549300DAUXQ2DCY4H838,14454,MA,25023.0,2.502350e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,7312,2.95,113700,132,2099,2345,43
3,2021,549300DAUXQ2DCY4H838,14454,MA,25023.0,2.502351e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,2380,77.98,113700,18,16,343,51
4,2021,549300DAUXQ2DCY4H838,14454,MA,25025.0,2.502513e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Ethnicity Not Available,...,NaN,NaN,NaN,5399,69.98,113700,54,568,1437,48


In [38]:
print(len(data1[data1['loan_purpose']==1]))

42655


In [8]:
# Number of Loans
count = 0
for i in range(data1.shape[0]):
  if pd.isna(data1.loc[i,'loan_purpose']) == False:
    count += 1
print(count)

165342


In [39]:
# Sum of Loans
sum = 0
for i in range(data1.shape[0]):
  if pd.isna(data1.loc[i,'loan_purpose']) == False:
    sum += data1.loc[i,'loan_amount']
print(sum)

72103510000.0


In [28]:
data1['loan_type'].head()

0    1
1    1
2    1
3    1
4    1
Name: loan_type, dtype: int64

In [42]:
tract_list = []
for i in range(data1.shape[0]):
  if data1.loc[i,'census_tract'] not in tract_list:
    tract_list.append(data1.loc[i,'census_tract'])

print(tract_list)
print(len(tract_list))


[25021402102.0, 25021403300.0, 25023503101.0, 25023510900.0, 25025130406.0, 25025110501.0, 25021420301.0, 25023510200.0, 25021442102.0, 25023504101.0, 25021403500.0, 25025040801.0, 25023505200.0, 25023530801.0, 25023542300.0, 25023510400.0, 25023510700.0, 25025140102.0, 25023540103.0, 25023542200.0, 25025120400.0, 25023511100.0, 25021416101.0, 25023520100.0, 25023505102.0, 25023510800.0, 25023502101.0, 25023544100.0, 25021411200.0, 25025082000.0, 25021420302.0, 25021413401.0, 25021414300.0, 25023511000.0, 25023511600.0, 25021415102.0, 25021402300.0, 25023526100.0, 25023511701.0, 25023510100.0, 25021421200.0, 25023503102.0, 25021456101.0, 25023504102.0, 25023510600.0, 25023525204.0, 25023511200.0, 25023521101.0, 25025170300.0, 25021402400.0, 25025020301.0, 25021411301.0, 25023521102.0, 25023506203.0, 25021404302.0, 25021420100.0, 25021419700.0, 25023524101.0, 25021456402.0, 25023506102.0, 25021412300.0, 25021412100.0, 25021423100.0, 25023525101.0, 25021402500.0, 25021406101.0, 250214135

In [ ]:
ans = []
for i in range(len(tract_list)):
  ans.append([0,0,0,0])

for i in range(len(tract_list)):
  for j in range(4):
    ans[i][j] = len(data1[(data1['census_tract'] == str(tract_list[i])) & (data1['loan_type'] == str(j+1))])


for i in range(len(tract_list)):
  if pd.isna(tract_list[i]):
    continue
  else:
    x = int(tract_list[i])
    print(x)
    print(str(ans[i][0])+' '+str(ans[i][1])+' '+str(ans[i][2])+' '+str(ans[i][3]))

In [44]:
loan_list = []
for i in range(len(tract_list)):
  loan_list.append(0)

for i in range(len(tract_list)):
  loan_list[i] = int(data1.loc[data1['census_tract'] == tract_list[i], 'loan_amount'].sum())

print(loan_list)
summ = 0
for i in range(len(loan_list)):
  summ += loan_list[i]
print(summ)

[157875000, 375440000, 371080000, 45765000, 77695000, 140815000, 223680000, 223900000, 252260000, 279855000, 392780000, 154720000, 473275000, 219820000, 165270000, 79845000, 180770000, 161650000, 154330000, 177990000, 259665000, 175530000, 380605000, 204640000, 286350000, 159500000, 152940000, 189430000, 295725000, 56100000, 222115000, 107575000, 202475000, 83195000, 199935000, 322785000, 203140000, 237355000, 170735000, 169445000, 182205000, 309170000, 172825000, 355375000, 112710000, 162310000, 119835000, 175260000, 239935000, 241585000, 15140000, 264560000, 105075000, 184325000, 251525000, 213030000, 168430000, 129870000, 190100000, 245760000, 302250000, 406955000, 684680000, 193875000, 183845000, 295835000, 95070000, 255155000, 84755000, 228235000, 253610000, 206805000, 98740000, 152565000, 242065000, 78035000, 87290000, 138445000, 261330000, 463480000, 263130000, 479250000, 99045000, 139485000, 224280000, 104355000, 139095000, 145625000, 210485000, 226440000, 34055000, 89755000, 2

In [34]:
!cp /content/drive/MyDrive/income-restricted-inventory-2021.csv .
df2 = pd.read_csv('income-restricted-inventory-2021.csv')
df2.describe()

,Zip Code,TtlProjUnits,RentUnits,OwnUnits,TtlMarket,MarketRent,MarketOwn,Total Income-Restricted,Income-Restricted Rental,Income-Restricted Ownership
count,1442.000000,1445.000000,1445.000000,1445.000000,1444.000000,1444.000000,1444.000000,1445.000000,1445.000000,1444.000000
mean,2125.432039,59.560554,50.979239,8.581315,20.247230,13.674515,6.572715,39.321107,37.319031,2.003463
std,13.627488,102.308222,100.132590,29.216307,63.463419,55.704068,27.671639,81.572228,82.166087,5.679777
min,2108.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
25%,2119.000000,5.000000,1.000000,0.000000,0.000000,0.000000,0.000000,3.000000,1.000000,0.000000
50%,2124.000000,23.000000,10.000000,0.000000,0.000000,0.000000,0.000000,10.000000,6.000000,0.000000
75%,2128.000000,66.000000,57.000000,1.000000,13.000000,0.000000,0.000000,42.000000,41.000000,1.000000
max,2215.000000,1100.000000,1100.000000,480.000000,736.000000,736.000000,479.000000,1100.000000,1100.000000,91.000000
